<h3>We try to find de subset of numbers that match with a sum</h3>

<h4>First import the data where is the sumands and the total</h4>

In [1]:
#some libraries
import pandas as pd
import numpy as np
from tqdm import tqdm
from numba import cuda, njit, prange
from itertools import chain, combinations

In [2]:
emitidas=pd.read_csv('Emitidas.csv')
integradas=pd.read_csv('integracion.csv')

In [3]:
emitidas.head()

,Nombre Cliente,RFC Cliente,Tipo Factura,Forma Pago,Moneda,Tipo Cambio,Mes,Folio Factura,UUID Factura,Fecha de Factura,Total Factura,Monto en Complemento de Pago,Saldo Abierto de Factura
0,"360 RECUPMATERIALES Y ACCESORIOS, S.A. DE C.V",TSR130304UP9,Ingreso,99,MXN,1,12,110408,B57A0C60-BA00-417B-8AAB-84E4A9E262E2,12/19/2019,1653.18,SIN COMPLEMENTO,1653.18
1,"360 RECUPMATERIALES Y ACCESORIOS, S.A. DE C.V",TSR130304UP9,Ingreso,99,MXN,1,12,110409,9F53BBB4-2F2B-4CC7-B946-DCB375F627A2,12/19/2019,351.04,SIN COMPLEMENTO,351.04
2,"360 RECUPMATERIALES Y ACCESORIOS, S.A. DE C.V",TSR130304UP9,Ingreso,99,MXN,1,12,110060,F8A8818E-688A-4CE3-9994-D0E9B9BB55BC,12/13/2019,1404.16,SIN COMPLEMENTO,1404.16
3,"360 RECUPMATERIALES Y ACCESORIOS, S.A. DE C.V",TSR130304UP9,Ingreso,99,MXN,1,12,110061,58987340-F843-4D77-AC2B-2BCABD521EB7,12/13/2019,1053.12,SIN COMPLEMENTO,1053.12
4,"360 RECUPMATERIALES Y ACCESORIOS, S.A. DE C.V",TSR130304UP9,Ingreso,99,MXN,1,12,109720,360FF10B-735A-42AA-BDF3-77839902A4B4,12/6/2019,1404.16,SIN COMPLEMENTO,1404.16


In [4]:
integradas.head()

,Cuenta Contable Cliente,RFC Cliente,Nombre Cliente,Saldos Iniciales,Notas de Credito,Facturas Emitidas,Neto,Pagos Identificados,Saldo Final,Suma de Facturas a Revisar,Comentario,Diferencia Saldo y facturas
0,105-01-20007,AAC080909QT7,ANTARES AGENCIA DE CREMACION S.A. DE C.V.,-,NaN,2550.00,2550.00,-,2550.00,2550.00,Saldo Identificado,0
1,105-01-10193,AAC961004FY2,ABBUD Y ASOCIADOS CONSTRUCCIONES SA DE CV,412035.09,1080000.0,12445133.89,11365133.89,11777168.98,0.00,0.00,Saldo Cero,NaN
2,105-01-10348,AACC660510HM3,JOSE CARLOS ALTAMIRANO CORRAL,3545.3,NaN,112961.51,112961.51,108652.29,7854.52,7854.52,Saldo Identificado,0
3,105-01-10357,AACL620923UH4,LINO EDUARDO ALVARADO CHICO,-,NaN,1499.60,1499.60,1499.6,0.00,0.00,Saldo Cero,NaN
4,105-01-10011,AAEM550521EG9,MARINA ALVAREZ ESPINOZA,7457.5,NaN,321669.71,321669.71,195807,133320.21,73699.32,NO REVISAR,59620.89


<h4>Filtering the rows that only have revision status in comments</h4>

In [5]:
revisar=integradas[integradas['Comentario']=='Revisar Facturas']

In [6]:
revisar.head()

,Cuenta Contable Cliente,RFC Cliente,Nombre Cliente,Saldos Iniciales,Notas de Credito,Facturas Emitidas,Neto,Pagos Identificados,Saldo Final,Suma de Facturas a Revisar,Comentario,Diferencia Saldo y facturas
14,105-01-10394,RPL960213CY7,RANCHO PRODUCTOR DE LECHE DE VILLA AHUMADA,419170.82,NaN,9.713830e+06,9.713830e+06,8383635.2,1749365.30,5876857.21,Revisar Facturas,4127491.91
15,105-01-10094,TLM7201085N4,TRANSPORTES LAR MEX SA DE CV,-,14669.00,1.810507e+08,1.810360e+08,179272184.58,1763863.13,4060898.58,Revisar Facturas,2297035.45
16,105-01-10423,FEM0504264R9,FLETES ESTRATEGICOS DE MEXICO SA DE CV,-,NaN,8.743466e+06,8.743466e+06,8248790.06,494675.83,1856468.26,Revisar Facturas,1361792.43
19,105-01-20045,TMA001209315,TORTILLERIA MARTINEZ SA DE CV,89950,9609.15,2.388882e+06,2.379273e+06,2422360.3,46862.30,597102.84,Revisar Facturas,550240.54
22,105-01-20037,GTN111215PI0,GAS TRADICIONAL DEL NORTE SA DE CV,1040957.99,NaN,3.026173e+06,3.026173e+06,3450733.48,616397.13,1144858.37,Revisar Facturas,528461.24


In [7]:
#function to search the posibly sumands
@njit
def find_subset_sum_dp(numbers, target):
    n = len(numbers)
    subset_sum = np.zeros((n + 1, int(target * 100) + 1), dtype=np.bool_)
    subset_sum[:, 0] = True  # The sum 0 can be achieved with any empty subset
    
    for i in range(1, n + 1):
        for j in range(1, int(target * 100) + 1):
            if int(numbers[i - 1] * 100) <= j:
                subset_sum[i, j] = subset_sum[i - 1, j] or subset_sum[i - 1, j - int(numbers[i - 1] * 100)]
            else:
                subset_sum[i, j] = subset_sum[i - 1, j]

    if not subset_sum[n, int(target * 100)]:
        return None

    # Reconstruct the subset
    subset = []
    i, j = n, int(target * 100)
    while i > 0 and j > 0:
        if subset_sum[i, j] and not subset_sum[i - 1, j]:
            subset.append(numbers[i - 1])
            j -= int(numbers[i - 1] * 100)
        i -= 1

    return subset

In [41]:
df_subsets = pd.DataFrame(columns=emitidas.columns)
print('Starting the process...')
for idx,row in revisar.iterrows():#runing over the rows of the revision information
    total=row['Saldo Final']#total of the sumands
    name=row['Nombre Cliente'] #namer of clien under revision
    data_user=emitidas[emitidas['Nombre Cliente']==name] #info of factures of client under revision
    sumands=[] #list for save the posibli sumands for the total
    for idxf,rowf in data_user.iterrows():#runing over the rows of the info of the user factures
        if rowf['Saldo Abierto de Factura']>0:
            sumands.append(rowf['Saldo Abierto de Factura'])#form a list of posibly sumands
    sumands= np.array(sumands, dtype=np.float64) 
    print('Searching for posibly sumands')
    subset=find_subset_sum_dp(sumands, total)#search the subset that give the sum
    print('filtering info')
    if subset is not None:
        for s in subset:
            condicion = (emitidas['Nombre Cliente'] == name) & (np.floor(emitidas['Saldo Abierto de Factura']) == int(s))
            ind=emitidas[condicion].index
            fila = emitidas.loc[ind]
            df_subsets = pd.concat([df_subsets, fila], ignore_index=True)

print('saving info')
df_subsets.to_csv('sumandos.csv', index=False)

Starting the process...
Searching for posibly sumands
filtering info
Searching for posibly sumands
filtering info
Searching for posibly sumands
filtering info
Searching for posibly sumands
filtering info
Searching for posibly sumands
filtering info
Searching for posibly sumands
filtering info
Searching for posibly sumands
filtering info
Searching for posibly sumands
filtering info
Searching for posibly sumands
filtering info
Searching for posibly sumands
filtering info
Searching for posibly sumands
filtering info
Searching for posibly sumands
filtering info
Searching for posibly sumands
filtering info
Searching for posibly sumands
filtering info
Searching for posibly sumands
filtering info
Searching for posibly sumands
filtering info
Searching for posibly sumands
filtering info
Searching for posibly sumands
filtering info
Searching for posibly sumands
filtering info
Searching for posibly sumands
filtering info
Searching for posibly sumands
filtering info
Searching for posibly sumands
f